In [1]:
from astropy.time import Time, TimeDelta

#import lsst.daf.butler as dafButler
from lsst_efd_client import EfdClient

In [2]:
# Summit
#dataPath = "/repo/LSSTComCam"
#efd_name = "summit_efd"
# RSP
#dataPath = "/repo/main"
efd_name = "ldf_stable_efd"

collections = ["LSSTComCam/raw/all"]
instrument = "LSSTComCam"
#butler = dafButler.Butler(dataPath, collections=collections, instrument=instrument)
client = EfdClient(efd_name)

In [304]:
obs_beg = Time("2021-06-04T12:00", scale="tai")
delta = TimeDelta(24*3600.0, format="sec", scale="tai")
obs_end = obs_beg + delta

In [305]:
start_int_df = await client.select_time_series("lsst.sal.CCCamera.logevent_startIntegration", "imageName",
                                               obs_beg, obs_end)

In [306]:
if start_int_df.size == 0:
    print("No images found!")
else:
    num_images = start_int_df.count()[0]

In [307]:
img_oods_df = await client.select_time_series("lsst.sal.CCArchiver.logevent_imageInOODS", "obsid,sensor",
                                              obs_beg, obs_end)

In [308]:
if img_oods_df.size == 0:
    print("No images in OODS!")

In [309]:
oods_images = img_oods_df.groupby("obsid").count()

In [310]:
bad_images = oods_images.index[oods_images.sensor != 9]

In [311]:
# Find images that have no sensors in OODS
missing_images = []
for image_name in start_int_df.imageName.array:
    if image_name not in oods_images.index:
        missing_images.append(image_name)

In [312]:
# Find images that do not have all 9 sensors in OODS
some_missing_images = []
standard_sensors = set(["S00", "S01", "S02", "S10", "S11", "S12", "S20", "S21", "S22"])
for image_name in bad_images:
    #print(image_name)
    sensors = img_oods_df.sensor[img_oods_df.obsid == image_name].array
    sensor_set = set([f"S{x}" for x in sensors])
    #print(sensor_set)
    missing_sensors = standard_sensors.difference(sensor_set)
    some_missing_images.append((image_name, sorted(missing_sensors)))

In [313]:
print("Summary")
print(f"Total images: {num_images}")
print(f"Images some missing sensors: {bad_images.size}")
print(f"Images missing all sensors: {len(missing_images)}")
print(f"Percentage of bad images: {(((bad_images.size + len(missing_images))/num_images)*100):.2f}%")
print("Images Missing All Sensors")
for image_name in missing_images:
    print(image_name)
print("Images Missing Some Sensors")
for image_name, missing_sensors in some_missing_images:
    sensor_str = ":".join(missing_sensors)
    print(f"{image_name}   {sensor_str}")

Summary
Total images: 218
Images some missing sensors: 12
Images missing all sensors: 1
Percentage of bad images: 5.96%
Images Missing All Sensors
CC_O_20210604_000037
Images Missing Some Sensors
CC_O_20210604_000005   S00:S01:S02
CC_O_20210604_000014   S00:S01:S02
CC_O_20210604_000023   S00:S01:S02
CC_O_20210604_000026   S00:S01:S02
CC_O_20210604_000030   S10:S11:S12
CC_O_20210604_000043   S00:S01:S02
CC_O_20210604_000084   S00:S01:S02
CC_O_20210604_000104   S00:S01:S02:S10:S11:S12
CC_O_20210604_000124   S00:S01:S02
CC_O_20210604_000134   S00:S01:S02
CC_O_20210604_000136   S00:S01:S02
CC_O_20210604_000169   S00:S01:S02
